<a href="https://colab.research.google.com/github/tovaalen/substrate_binding_GumK/blob/main/Docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Molecular docking to generate an ensemble of docked poses

This google colab is written to generate an ensemble of docked poses using the docking program GNINA (https://github.com/gnina/gnina).


Upload the needed strucutral files (in pdb format) and add the necessary information in the cell below, before running all the cells.


The distance between C6 of the sugar of the ligand and residue 307 of the protein, and the dihedrals, will only be calculated if the ligand file is named 'udp_glcA.pdb' or 'udp_glc.pdb' (e.i. the ligand is UDP-GlcA or UDP-Glc)

In [ ]:
#@title Add all information about the docking, then run all cells

#import re
#import hashlib
#import random

#from sys import version_info
#python_version = f"{version_info.major}.{version_info.minor}"

#def add_hash(x,y):
#  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

jobname = 'test' #@param {type:"string"}
#@markdown **protein_names** and **protein_files**: The name of the protein and the filename of the protein file. Note: you can only dock to one protein at a time.
protein_name = 'WT conformation 1' #@param {type:"string"}
protein_file = 'donor_domain.pdb' #@param {type:"string"}
#@markdown **protein_atom**: The number of the protein atom to calc the distance to C6 of the sugar when docking UDP-GlcA and UDP-Glc. Can be found in the PDB file.
protein_atom = 1356 #@param {type:"integer"}
#@markdown **ligand_names** and **ligand_files**: The name of the ligands and corresponding structural file. If you want to dock multiple different ligands, seperate the ligand names and file names with a comma (note: any spaces in the name will be removed)
ligand_names = 'UDP-GlcA, UDP, UDP-Glc' #@param {type:"string"}
ligand_files = 'udp_glcA.pdb, udp.pdb, udp_glc.pdb' #@param {type:"string"}
#@markdown **autobox:** the filename of the autobox to use
autobox = 'WT1_autobox.pdb' #@param {type:"string"}
#@markdown **calc_rmsd:** check if you want to calculate the RMSD to a reference file. If so, provide the file name of the reference strucutre in **reference_structure**
calc_rmsd = True #@param {type:"boolean"}
reference_structure = 'crystal_uridine.pdb' #@param {type:"string"}

#@markdown **download:** check if you want the files to be downloaded (e.i. the sdf file with the docked ligands and an excel sheet with the corresponding scores and calculated attributes)
download = True #@param {type:"boolean"}

#@markdown Number of docking reruns (the top 20 poses from each run are saved)
reruns = 10 #@param {type:"integer"}

ligand_names = [word.strip() for word in ligand_names.split(',')]
ligand_files = [word.strip() for word in ligand_files.split(',')]

### Setup

In [ ]:
import sys

''' functions used in the script '''

def atom_pos(mol, i):
  pos = mol.GetConformer().GetAtomPosition(i)
  p = [pos.x,pos.y,pos.z]
  return np.array(p)

def is_module_imported(module_name):
    return module_name in sys.modules

In [ ]:
# import all packages needed, if not already done
if not is_module_imported('rdkit'):
  for i in range(2):
    !apt install openbabel --quiet
    from google.colab import files
    import os
    import shutil
    import pandas as pd
    !pip install rdkit-pypi
    !wget https://downloads.sourceforge.net/project/smina/smina.static
    !wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina
    !chmod +x gnina # make executable

    !pip install MDAnalysis
    import MDAnalysis as mda
    from MDAnalysis.lib import distances
    import math

  import rdkit
  from rdkit import Chem
  from rdkit.Chem import SDWriter
  import os
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  import shutil
  import numpy as np


## Docking

In [ ]:
runs = range(1, reruns+1)
id = range(1, reruns*20+1)

# prepare all files
protein_mol = Chem.MolFromPDBFile(protein_file, removeHs=False)

os.mkdir(jobname)

new_path = os.path.join(jobname, 'protein.pdb')
shutil.copy(protein_file, new_path)

new_path = os.path.join(jobname, 'autobox.pdb')
shutil.copy(autobox, new_path)

if reference_structure is not None:
  new_path = os.path.join(jobname, 'reference_structure.pdb')
  shutil.copy(reference_structure, new_path)

os.chdir(jobname)

for name, filename in zip(ligand_names, ligand_files):
  os.mkdir(name)
  old_path = os.path.join('..', filename)
  new_path = os.path.join(name, 'ligand.pdb')
  shutil.copy(old_path, new_path)

!obabel protein.pdb -p 7 -xr -Oprotein.pdb #protonation at pH 7, -xr means keep rigid

In [ ]:
for name, filename in zip(ligand_names, ligand_files):
  os.chdir(name)

  !obabel ligand.pdb -Oligand.pdb

  all_poses = []
  scores_df = pd.DataFrame()

  for r in runs:
    # run docking with gnina
    !../../gnina -r ../protein.pdb -l ligand.pdb --autobox_ligand ../autobox.pdb -o docked_ligands.sdf.gz --exhaustiveness 16 --autobox_extend 1 --min_rmsd_filter 0.1 --num_modes 20

    #unzip file with docked poses
    !gunzip docked_ligands.sdf.gz

    # rescore with Vinardo
    !../../gnina --score_only -r ../protein.pdb -l docked_ligands.sdf --scoring vinardo -o rescored_vinardo.sdf --quiet

    # calculate RMSD
    if calc_rmsd:
      rmsd = []
      !obrms -f ../reference_structure.pdb docked_ligands.sdf > RMSD.txt
      with open('RMSD.txt', 'r') as file:
        lines = file.readlines()
        for line in lines:
          rmsd.append(float(line.split(':')[1].strip()))
      os.remove('RMSD.txt')
    else:
      rmsd = ['NA'] * 20

    # add docked poses to list with all poses
    supplier_poses = Chem.SDMolSupplier('docked_ligands.sdf')
    for mol in supplier_poses:
      all_poses.append(mol)
    os.remove('docked_ligands.sdf')

    supplier_poses_vinardo = Chem.SDMolSupplier('rescored_vinardo.sdf')
    os.remove('rescored_vinardo.sdf')

    #coordinates of atoms in protein needed to calc distance
    a = protein_mol.GetConformer().GetAtomPosition(protein_atom - 1) # -1 because rdkit starts fr 0
    p_protein = [a.x,a.y,a.z]

    dist = []
    dihedral1 = []
    dihedral2 = []

    # If the ligand is UDP-GlcA or UDP-Glc => calc distance and dihedral angles
    if filename == 'udp_glcA.pdb' or filename == 'udp_glc.pdb':
      if filename == 'udp_glcA.pdb':
        C6_atom = 36 # atom number for C6 atom in sugar, to calc distance
        dihedral1_atoms = [32, 24, 21, 20] # atoms to calc dihedral angles
        dihedral2_atoms = [25, 32, 24, 21]
      else:
        C6_atom = 35 # atom number for C6 atom in sugar, to calc distance
        dihedral1_atoms = [33, 24, 21, 20] # atoms to calc dihedral angles
        dihedral2_atoms = [25, 33, 24, 21]
      for mol in supplier_poses:
        # calc distance
        pos = mol.GetConformer().GetAtomPosition(C6_atom)
        p_C6 = [pos.x,pos.y,pos.z]
        dist.append(math.dist(p_C6,p_protein))

        # calc dihedral 1
        a1 = atom_pos(mol, dihedral1_atoms[0])
        a2 = atom_pos(mol, dihedral1_atoms[1])
        a3 = atom_pos(mol, dihedral1_atoms[2])
        a4 = atom_pos(mol, dihedral1_atoms[3])
        angle1 = distances.calc_dihedrals(a1,a2,a3,a4)
        dihedral1.append(math.degrees(angle1))

        # calc dihedral 2
        a1 = atom_pos(mol, dihedral2_atoms[0])
        a2 = atom_pos(mol, dihedral2_atoms[1])
        a3 = atom_pos(mol, dihedral2_atoms[2])
        a4 = atom_pos(mol, dihedral2_atoms[3])
        angle2 = distances.calc_dihedrals(a1,a2,a3,a4)
        dihedral2.append(math.degrees(angle2))

    else:
      # if the ligand is not UDP-GlcA or UDP-Glc, no dihedral angles or distances are calculated
      dist = ['NA'] *20
      dihedral1 = ['NA'] *20
      dihedral2 = ['NA'] *20

    scores = []

    for mol1, mol2, line, d, dh1, dh2 in zip(supplier_poses, supplier_poses_vinardo, rmsd, dist, dihedral1, dihedral2):

      scores.append({
          'Protein': protein_name,
          'Substrate': name,
          'Run': r,
          'CNNscore': float(mol1.GetProp('CNNscore')),
          'CNNaffinity': float(mol1.GetProp('CNNaffinity')),
          'Vina': float(mol1.GetProp('minimizedAffinity')),
          'Vinardo': float(mol2.GetProp('minimizedAffinity')),
          'RMSD': line, #float(line.split(':')[1].strip()), ska ev vara float(line.split(' ')[2].strip())
          'distance' : d,
          'dihedral1' : dh1,
          'dihedral2' : dh2
          })

    scores_df = pd.concat([scores_df, pd.DataFrame(scores)], ignore_index=True)

  scores_df.insert(3, 'id', id)
  scores_df.to_excel('score.xlsx', index=False)

  sdf_writer = Chem.SDWriter('docked_poses.sdf')
  for i, mol in zip(id, all_poses):
    if mol is not None:
      mol.SetProp('_Name', str(i)) # Sets the name to the id number
      sdf_writer.write(mol) # Append the molecule to the SDF file

  os.chdir('..')

if download:
  #download files
  !zip -r download.zip *
  os.chdir('..')
  old_zip_file = jobname + '/download.zip'
  new_zip_file = jobname + '.zip'
  os.rename(old_zip_file, new_zip_file)
  files.download(new_zip_file)